## 하이퍼파라미터 튜닝

#### 파라미터(parameter)

- 학습 과정에서 생성되는 변수들
- 머신러닝이나 딥러닝 모델에서 생성된 가중치,계수

#### 하이퍼 파라미터(hyper parameter)

- 모델을 생성할 때, 사용자가 직접 설정하는 변수
- 랜덤 포레스트에서 max_depth(트리의 깊이),n_estimators(트리의 갯수),로지스틱 회귀에서 C(규제의 정도 c가 작을수록 규제가 커짐),penealty(규제 종류),max_iter(반복 횟수)
- 딥러닝 모델에서 layer의 개수 epoch(학습횟수)의 갯수 등이 해당됨

## GridSearchCV

- 모델링시 필요한 하이퍼파라미터를 설정할 때 가장 최적의 파라미터값을 찾아주는 방법 중 하나로써 파라미터 후보값들을 사전에 정의해주고 이를 일일히 돌려서 적용해보는 것을 대신 해주는 방법이다.

- 성능을 개선 하기 용이 하지만 모든 조합의 성능을 확인해야 하다 보니 시간이 오래 걸리는 단점이 있다

#### GridSearchCV 과정

1. 지정할 매개변수와 값을 지정(dictionary 형태)

2. train set에서 그리드 서치를 수행해 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾기(best_estimator_)

3. 이 매개변수로 전체 train set으로 최종 모델을 train



## RandomizedSearchCV

- 분류기(Esimator)를 결정하고 해당 분류기의 최적의 하이퍼 파라미터를 찾기 위한 방법 중 하나이다.

- 주어진 문제에 대한 분류기들로 모델을 작성한 뒤, 성능 개선을 위한 Tuning을 하는데 일일히 모든 파라미터를 다 조율해보고, 그에 맞는 최적의 조합을 찾아보긴 힘들기 때문에, 오차값이 가장 적은 하이퍼파라미터를 찾아주는 좋은 라이브러리이다.


## Pipeline

- 사이킷런의 from sklearn.pipeline import make_pipeline으로 쓸 수 있는 기능으로 pipeline에 학습하고자 하는 것을 한 번에 집어 넣고 pipe를 통해 한 번에 학습을 적용할 수 있다.

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib as mpl
import scipy.stats as stats 

from IPython.display import set_matplotlib_formats

mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus =True)
set_matplotlib_formats('retina')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
import warnings 
warnings.filterwarnings('ignore')


# 학습/검증 데이터 분할 / 매개변수 튜닝 / 교차검증 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# 평가 
from sklearn.metrics import classification_report


C:\Users\user\AppData\Local\Temp/ipykernel_15152/2949398509.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
wine = pd.read_csv('https://bit.ly/wine-date')

In [4]:
data = wine.iloc[:,:3].to_numpy()
target=wine.iloc[:,3].to_numpy()

In [5]:
## 데이터 셋 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [6]:
X_sub,X_valid,y_sub,y_valid=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_sub,y_sub)
print(dt.score(X_sub,y_sub))
print(dt.score(X_valid,y_valid))

0.9971133028626413
0.864423076923077


In [9]:
## 교차 검증 
from sklearn.model_selection import cross_validate
scores =cross_validate(dt,X_train,y_train)
print(scores)
#검증 fold의 score
print(np.mean(scores['test_score']))

{'fit_time': array([0.01022363, 0.01295161, 0.00996995, 0.00704241, 0.00502872]), 'score_time': array([0.0019989 , 0.00101352, 0.        , 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,X_train,y_train,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=15,shuffle=True,random_state=42)
scores = cross_validate(dt,X_train,y_train,cv=splitter)
print(np.mean(scores['test_score']))

0.8626087632500986


### GridSearchCV

In [12]:
# 하이퍼파라미터 min_impurity_decrease 값 0.0001~0.0005로 범위조정
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [13]:
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
dt = gs.best_estimator_
print(dt)
print(gs.best_params_)
print(gs.best_score_)
print(dt.score(X_train,y_train))

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}
0.8681929740134745
0.9615162593804117


In [21]:
#HyperParameter 성능 검증 : min_impurity_decrease가 0.0001일때 성능 가장 좋음
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [23]:
#HyperParameterTuning
#min_impurity_decrease: 0.0001 부터 0.001까지 0.0001씩 증가
#max_depth:5부터 20까지 1씩 증가
#min_samples_split:2부터 100까지 10씩 증가
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
       'max_depth':range(5,20,1),
       'min_samples_split':range(2,200,10)}

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [25]:
gs.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 200, 10)})

In [27]:
#최적 파라미터 모델
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### RandomSearchCV

- randint() - 정수를 주어진 범위에서 고르게 추출(균등 분포에서 샘플링)
- uniform() - 실수를 주어진 범위에서 고르게 추출

In [29]:
from scipy.stats import uniform,randint

In [30]:
rgen=randint(0,10)
rgen.rvs(10)

array([8, 5, 1, 5, 2, 1, 5, 4, 3, 9])

In [31]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  92, 100, 108,  95, 113,  98,  89, 102, 106], dtype=int64))

In [33]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.38360773, 0.78207163, 0.56798758, 0.58274982, 0.09748544,
       0.99798651, 0.57413934, 0.82527607, 0.8385827 , 0.63242717])

In [34]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
       'max_depth':randint(14,60),
       'min_samples_split':randint(2,25),
       'min_samples_leaf':randint(2,25)}

In [35]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                      b n_iter=77,n_jobs=-1,random_state=42)

In [36]:
rs.fit(X_train,y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_iter=77,
                   n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC5BBE52B0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC5BBB8EE0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC5BBE59A0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC5BBE4B80>},
                   random_state=42)

In [37]:
print(rs.best_estimator_)

DecisionTreeClassifier(max_depth=35,
                       min_impurity_decrease=0.0003439896433790836,
                       min_samples_leaf=7, min_samples_split=7,
                       random_state=42)


In [38]:
print(rs.best_params_)

{'max_depth': 35, 'min_impurity_decrease': 0.0003439896433790836, 'min_samples_leaf': 7, 'min_samples_split': 7}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [40]:
rs_score=rs.best_estimator_
print(rs_score.score(X_test,y_test))

0.86


## Ensemble(앙상블)

![ensemble](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbre2VP%2FbtqFnu91zIX%2F9TCrTQOcoyKxFp9lJxYuXk%2Fimg.png)

- 앙상블 학습(Ensemble Learning)은 여러 개의 분류기를 생성하고, 그 예측을 결합함으로써 보다 정확한 예측을 도출하는 기법을 말한다.

- 성능이 강력한 모델을 하나 사용하는 대신 성능이 약한 모델 여러개를 조합하여 보다 더 정확한 예측에 대한 도움을 주는 방식

- 앙상블 기법의 예측력을 평가하기 위해서는 단일 모델의 예측을 평가하는 것보다 더 많은 계산을 필요로 한다.

![bv](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FvhlZ1%2FbtqyfOOjKul%2FhyJrHX8TmrBY8uJvcQVoo0%2Fimg.png)

#### bias(편향)

- 예측값이 실제 모델값과 얼마나 떨어져 있는지 측정한 값(제곱 에러)
- 모델이 지나치게 단순하면 편향이 높아질 수 있다.(underfitting)

#### variance(분산)

- 예측 데이터가 서로 얼마나 흩어져 있는지 측정한 값
- 모델이 지나치게 복잡하거나 ovefitting이 된 경우 커짐

#### bias-variance

- bias가 높고 variance가 낮은 경우
  - 모델의 결과는 안정적이나 잘못된 값으로 예측 값이 나오기 때문에 모델을 더 복잡하게 만들거가 최적화를 더 깊게 해야함

- bias가 낮고 variance가 높은 경우
  - 모델이 정확하게 매핑 되지만 과적합 되어 데이터가 조금만 변해도(노이즈) 모델이 
    아예 다른 결과가 나오기 때문에 feature 수를 줄이거나 규제를 사용한다

#### Bootstrap

- 샘플 데이터 셋에서 중복을 허용하여 무작위로 임의 샘플셋을 만들고, 이를 통해 통계값을 산출해, 모집단 또는 주어진 샘플 데이터 셋 보다 큰 데이터에 대한 분포등을 추정 하는 것



### 보팅(Voting)

- 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식
- 서로 다른 알고리즘을 여러개 결합해 사용(Svm,Logistic Regression..)

__하드 보팅(Hard Voting)__

- 다수의 분류기가 예측한 결과값을 최종 결과로 선택

__소프트 보팅(Soft Voting)__

- 모든 분류기가 예측한 레이블 값의 결정 확률 평균을 구한 뒤 가장 확률이 높은 레이블 값을 최종 결과로 선정

###  배깅(Bagging)

![bagg](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb4wG8O%2FbtqyfYW98AS%2FYZBtUJy3jZLyuik1R0aGNk%2Fimg.png)

- Bagging은 Bootstrap Aggregating의 줄임말
- Bootstrap 기법을 사용해 조금씩 서로 다른 train set을 생성해 모델(train된 트리)을 생성하고 결과를 결합(aggregating)하는 방법
- 각 샘플의 결과변수가 연속형일 경우 평균, 범주형일 경우 다중 투표(majority vote)를 사용 
- Bagging 기법을 사용하는 알고리즘으로 Random Forrest,Extra Tree등이 있다

#### OOB(Out-of-Bag)
![oob](https://editor.analyticsvidhya.com/uploads/90661letsee.jpg)
- 배깅(bagging)알고리즘에서 수가 일정 수준 이상으로 늘어나면 데이터의 2/3 정도만 bootstrap에 샘플링 되고 나머지 샘플링 되지않은 1/3의 데이터들을 OOB(Out-of-Bag)이라고 한다.
- OOB는 검증용 데이터 셋으로 사용되며 배깅 알고리즘 함수에서 oob_score를 true로 하면 훈련 종료후 oob 샘플을 기반으로 성능 평가를 수행한다. 

### 부스팅(Boosting)

![boost](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FkCejr%2FbtqyghvqEZB%2F9o3rKTEsuSIDHEfelYFJlk%2Fimg.png)

- Boosting은 전체 학습 데이터를 기반으로 분류기를 구성하여 예측을 수행한 후, 예측이 잘못된 데이터에 가중치를 더 두어 약 분류기를 강 분류기로 만드는 방법

- 분류 모델이 각각 값을 예측하는 Bagging과는 달리 Boosting은 분류 모델이 서로 조화를 이룬다. 처음 모델이 예측읋 하면 그 예측 결과에 따라 데이터에 가중치가 부여되고, 부여된 가중치가 다음 모델에 영향을 준다.

- 예를 들어 각 데이터마다 가중치(sample weight) 컬럼을 두고, 최초에는 동일한 가중치를 갖게함 -> 최초 약 분류기(weak larner)를 통해 오류 데이터의 가중치를 높임 -> 이를 기반으로 다음 분류기에 학습시킬 train set에 해당 오류 데이터가 들어갈 확률을 높임(직전 분류기의 eroor를 반영해 현재 분류기의 eroor를 잡겠다는 것) 

- 이를 통해 n개의 모델을 진행해, 모델별 가중치(weight) x 각 분류기의 합으로 최종 예측값을 결정

- 예측이 틀린 데이터에 대해 예측값을 보정하는 것을 반복해나가며 모델 성능을 개선(bias 감소)

- Boosting 기법을 사용하는 알고리즘으로 AdaBoost,GBM,XGBoost,LightGBm,HistGBM,CatBoost등이 있다.

## 스태킹(Stacking)

![stack](https://media.vlpt.us/images/dbj2000/post/8cf8374b-4ea6-437d-a0b3-92276af08abc/image.png)

- 스태킹 기법은 다른 앙상블 기법인 배깅과 부스팅처럼 여러 알고리즘들을 결합해 예측 결과를 도출한다

- 다른 두 기법과의 차이점은 개별 알고리즘(svm이나 knn)으로 예측한 데이터를 기반으로 다시 예측을 수행해 예측 모델을
 만든다는 차이점이 있다

- 개별 알고리즘의 예측 결과 데이터 셋을 최종적인 메타 데이터 셋으로 만들어 별도의 알고리즘을 생성, 최종 학습을 수행
하고 다시 테스트 데이터를 기반으로 최종적으로 예측

   - 메타 데이터: 개별 데이터의 예측된 데이터 셋을 다시 기반으로 학습하고 예측해 만들어진 데이터

- 스태킹 모델의 핵심은 여러 개별 모델의 예측 데이터를 각각 스태킹 기법으로 결합해 최종 메타 모델의 train, test set을 
만드는 것

- 캐글이나 데이콘에서 조금이라도 높은 순위를 따내기 위해 많이쓰는듯

[스태킹 앙상블 순서]

1. 모델별로 각각 학습 시킨 뒤 예측을 수행하면 각각 M개의 로우를 가진 1개의 레이블 값을 도출할 것임
2. 모델별로 도출된 예측 레이블 값을 다시 합해서(스태킹) 새로운 데이터 세트를 만듦
3. 스태킹 된 데이터 세트에 대해 최종 모델을 적용해 최종 예측

### 랜덤 포레스트(Random Forrest)

![rfc](https://itwiki.kr/images/thumb/1/12/%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.png/750px-%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.png)

- random forrest(랜덤 포레스트) 알고리즘은 decision tree(의사결정 나무) 모델을 여러 개 훈련시켜 그 결과를 종합해서 예측하는 알고리즘이다.

- 각 의사결정 나무 모델을 훈련시킬 때 배깅(bagging) 방식을 사용해 일반화(안정성) 성능을 향상 시킨다.

- 데이터 scaling을 할 필요가 없으며 과적합이 잘 되지 않는다는 장점이 있지만 차원이 크고 희소한 데이터(text data)에서 성능이 미흡하며 훈련시 메모리 소모가 크며 train set을 추가해도 성능 개선이 어려운 단점이 있다.

- 주어진 샘플에서 모든 feature에 대한 정보 이득(information gain)을 계산해 가장 높은 정보 이득을 가지는(설명력이 가장 높은) 변수를 split node로 설정한다.

### 기존의 부스팅 기법

1. 실제 데이터들의 mean과 residual를 구한다.

2. 데이터로 이 residual을 학습하는 모델을 만든다.

3. 만들어진 모델로 예측하여, 예측 값에 Learning_rate를 곱해 실제 예측 값(평균+잔차예측 값*lr)을 업데이트 한다.

4. 1~3 반복

 

### 문제점
#### 느린 학습 속도
- boosting model이 아닌 배깅과 비교했을 때 훨씬 느린 속도를 보인다. 배깅의 경우 여러 트리들이 병렬적으로 모델학습을 수행하고 부스팅의 경우 순차적으로 모델학습을 수행한다. 
  
  그래서 새롭게 나오는 부스팅 모델들(XGBoost, Light GBM, Catboost 등)의 알고리즘에는 이런 속도 문제 해결도 포함한다. 

 

#### 오버피팅
- 속도 문제는, 샘플링이나 알고리즘 최적화로 어느정도 개선되는 반면, 오버피팅은 그렇지 않다. 이는 부스팅이라는 개념 자체가 가지고 있는 문제인데, 위 과정만 보더라도 왜 오버피팅이 일어나는지 알 수 있다.

  애초, 부스팅 자체가 'residual'을 줄여나가기 위해 학습하는 모델이기 때문에 굉장히 High Variance한 모델일 수 밖에 없다.

  이에, 새롭게 나오는 부스팅 모델들의 대부분은 이 오버피팅을 해결하기 위한 기법들을 알고리즘에 포함시킨다.

 

## Catboost

#### Level-wise Tree
- XGBoost와 더불어 Catboost는 Level-wise로 트리를 만들어나가는 반면 Light GBM은 Leaf-wise이다.

- Level-wise와 Leaf-wise의 차이는, 직관적으로 Level-wise는 BFS(너비 우선 탐색:Breath-first search)  같이 트리를 만들어나가는 형태이고,

- Leaf-wise는 DFS(깊이 우선 탐색: Depth-first search) 같이 트리를 만들어나가는 형태이다. 물론 max_depth=-1이면 둘다 같은 형태지만, 대부분의 부스팅 모델에서는 트리는 max_depth!=-1이기 때문에 이 둘을 구분하는 것이다.

 

#### Ordered Boosting
- Catboost는 기존의 부스팅 과정과 전체적 흐름은 비슷하되 약간 다르다. 

- 기존의 부스팅 모델이 일괄적으로 모든 훈련 데이터를 대상으로 잔차를 계산했다면, Catboost는 일부만 가지고 잔차계산을 한 뒤, 이것을 통해 모델을 만들고, 그 후, 데이터의 잔차는 이 모델로 예측한 값을 사용한다. 




기존 부스팅 기법은 모든 datapoint(x1-x10)까지의 잔차를 일괄 계산한다.

반면, Catboost의 과정은 다음과 같다.

1. 먼저 x1의 잔차만 계산하고, 이를 기반으로 모델을 만든다. 그 후, x2의 잔차를 이 모델로 예측한다.

2. x1,x2의 잔차를 가지고 모델을 만든다. 이를 기반으로 x3,x4의 잔차를 모델로 예측한다.

3. x1,x2,x3,x4의 잔차를 가지고 모델을 만든다. 이를 기반으로 x5,x6,x7,x8의 잔차를 모델로 예측한다.

4. 이 과정을 반복한다.

 

#### 이렇게 순서에 따라 모델을 만들고 예측하는 방식을 Ordered Boosting이라 부른다.

 

#### Random Permutation
- Ordering Boosting을 할 때, 데이터 순서를 randomize 하지 않으면 매번 같은 순서대로 예측하는 모델을 만들 가능성이 있다. 이 순서는 사실 우리가 임의로 정한 것이므로, 순서 역시 매번 섞어줘야 한다. Catboost는 이러한 것 역시 감안해서 데이터를 shuffling하여 뽑아낸다. 뽑아낼 때 역시 모든 데이터를 뽑는게 아니라, 그 중 일부만 가져오게 할 수 있다.

- 이 모든 기법이 다 Overfitting을 방지하기 위해, 트리를 다각적으로 만드려는 시도이다.

 

- Ordered Target Encoding
Target Encoding, Mean Encoding, Response Encoding 이라고 불리는 기법을 사용한다. (모두 같은 말)

범주형 변수를 정수로 인코딩 시키는 방법 중, 가장 최근에 나온 기법인데, 간단히 설명하면 다음과 같다.


위 데이터에서 time, feature1로 class_label을 예측해야한다고 해보자.

feature1의 cloudy는 다음과 같이 인코딩 할 수 있다.

cloudy=(15+14+20+25)/4=18.5

즉, cloudy를 cloudy를 가진 데이터들의 class_label의 값의 평균으로 인코딩 하는 것이다. 이 때문에 Mean encoding이라 불리기도 한다.

그런데 위는 우리가 예측해야하는 값이 train_set feature에 들어가버리는 Data Leakage문제를 일으킨다. 이는 오버피팅을 일으키는 주 원인이자, Mean encoding 방법 자체의 문제이기도 하다.

Catboost는 이에대한 해결책으로, 현재 데이터의 인코딩하기 위해 이전 데이터들의 인코딩된 값을 사용한다.

 

-Friday, cloudy=(15+14)/2=15.3 encoding

-Saturday, cloudy=(15+14+20)/3=16.3 encoding

 

즉, 현재 데이터의 타겟 값을 사용하지 않고, 이전 데이터들의 타겟 값만을 상요하니, Data Leakage가 일어나지 않는다.

범주형 변수를 정수 인코딩할 때, 오버피팅도 막고 수치 값의 다양성도 만들어주는 영리한 기법이다.

이러한 시도는 Smoothing, Expanding 등이 있었는데, 이 시도 역시 이런 종류 중 하나라고 볼 수 있다.

 

#### Categorical Feature Combinations

- 다음 예를 보면, country 변수만 봐도 hair_color feature가 결정되기 때문에, class_label을 예측하는데 있어, 두 feature가 전부 필요하지 않고 이 중 하나의 feature만 있으면 된다. Catboost는 이렇게, information gain이 동일한 두 feature를 하나의 feature로 묶어버린다. 결과적으로, 데이터 전처리에 있어 feature selection부담이 조금 줄어든다고 할 수 있다.

 

#### One-hot Encoding

- 범주형 변수를 항상 Target Encoding하는 것은 아니다. Catboost는 낮은 Cardinality를 가지는 범주형 변수에 한해서, 기본적으로 One-hot encoding 을 실시한다. Cardinality 기준은 one_hot_max_size 파라미터로 설정할 수 있다.

- 예를 들어, one_hot_max_size=3 으로 준다면, Cardinality가 3 이하인 범주형 변수들은 Target Encoding이 아니라 One_hot으로 인코딩 된다.

  아무래도 Low Cardinality를 가지는 범주형 변수의 경우 Target Encoding 보다 One-hot이 더 효율적이기 때문이다.

 

#### Optimized Parameter tuning
- Catboost는 디폴트 파라미터가 기본적으로 최적화가 잘 되어있어, 파라미터 튜닝에 크게 신경쓰지 않아도 된다고 한다.

  반면 xgboost나 light gbm은 파라미터 튜닝에 매우 민감하다. 사실 대부분 부스팅 모델들이 파라미터 튜닝하는 이유는, 트리의 다형성과 오버피팅 문제를 해결하기 위함인데, Catboost는 이를 내부적인 알고리즘으로 해결하고 있으니 굳이 파라미터를 튜닝할 필요가 없다는 것이다.

  굳이 해야한다면, learning_rate, random_strength, L2_regulariser과 같은 파라미터 튜닝인데 큰 차이는 없다고 한다.

 

 

### Catboost의 한계
- Sparse한 Matrix는 처리하지 못한다.

- 데이터의 대부분이 수치형 변수인 경우, Light GBM보다 학습 속도가 느리다. 즉, 대부분이 범주형 변수인 경우 쓰라는 것이다.

 

In [1]:
wine = pd.read_csv('https://bit.ly/wine-date')

NameError: name 'pd' is not defined

In [60]:
data = wine.iloc[:,:3].to_numpy()
target=wine.iloc[:,3].to_numpy()

In [61]:
## 데이터 셋 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [62]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [63]:
rf = RandomForestClassifier(n_estimators=10,n_jobs=-1,random_state=42)

In [68]:
splitter = StratifiedKFold(n_splits=4,random_state=42,shuffle=True)
scores = cross_validate(rf,X_train,y_train,return_train_score=True,n_jobs=-1,cv=splitter)

In [69]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9902506868434797 0.8801239118848818


In [70]:
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=42)

In [71]:
# 변수 중요도 --> decision tree에 비해 균일함
print(rf.feature_importances_)

[0.22029335 0.50534665 0.27436   ]


In [72]:
#oob score
rf= RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)

In [73]:
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [74]:
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [76]:
print(rf.oob_score_)

0.8934000384837406


### 엑스트라 트리(Extra Tree)

- 엑스트라 트리(Extra Tree)는 랜덤 포레스트(Random Forrest)와 달리 부트스트랩을 하지않고 원본 데이터를 그대로 가져다 쓴다.(비복원추출)

- 엑스트라 트리는 트리를 split할 때 무작위로 feature를 선정한다. 모든 feature에 대한 정보 이득(information gain)을 계산해 설명력이 가장 높은 변수를 split node로 설정하는 랜덤 포레스트와는 달리 아무 feature나 고른 다음 그 feature에 대해서만 최적의 node를 찾아 분할한다. 

- 엑스트라 트리는 랜덤 포레스트에 비해 모델 자체의 성능은 낮지만 속도가 훨씬 빠르며 overfitting을 막고 valid set의 점수를 높이는 효과가 있다.

In [77]:
from sklearn.ensemble import ExtraTreesClassifier

In [80]:
et = ExtraTreesClassifier(n_estimators=10,n_jobs=-1,random_state=42)
splitter = StratifiedKFold(n_splits=15,random_state=42,shuffle=True)
scores = cross_validate(rf,X_train,y_train,return_train_score=True,n_jobs=-1,cv=splitter)

In [81]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9970449965678243 0.8943690204505449


In [82]:
et.fit(X_train,y_train)

ExtraTreesClassifier(n_estimators=10, n_jobs=-1, random_state=42)

In [83]:
print(et.feature_importances_)

[0.20301195 0.52499305 0.271995  ]


### 그레이디언트 부스팅(Gradient Boosting)

![gbm](https://ichi.pro/assets/images/max/724/1*pEu2LNmxf9ttXHIALPcEBw.png)
- 경사 하강법을 사용해, Boosting 기법을 수행하는 알고리즘
- train set의 feature 값을 기반으로 잔차(residual)를 줄여나가기 위해, 잔차를 대상으로 예측 이를 위해 경사 하강법 사용
- 이전 트리의 오차를 강하게 보정하기 위해 학습률(learning rate)을 사용한다.
- 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 MSE를 사용
- 예측 값 + 학습률 x 잔차(모델 1) +학습률 x 잔차(모델 2)...로 진행됨
- 최종값은 예측 값 + 잔차와 비슷한 수치가 된다.
- 결정트리를 계속 추가해가면서 천천히 조금씩 이동해야 하기 때문에 사전 가지치기를 하거나 깊이가 얕은 트리를 사용해야한다. 

#### 잔차(residuel)

- 표본집단(sample)에서 예측값(초기는 평균)과 실제값의 차이

In [85]:
from sklearn.ensemble import GradientBoostingClassifier

In [129]:
gb = GradientBoostingClassifier(n_estimators=50,learning_rate=0.002,random_state=42)

In [130]:
splitter = StratifiedKFold(n_splits=110,random_state=42,shuffle=True)
scores = cross_validate(rf,X_train,y_train,return_train_score=True,n_jobs=-1,cv=splitter)

In [131]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9969301258507083 0.8922308188265635


In [132]:
gb.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.002, n_estimators=50,
                           random_state=42)

In [133]:
print(gb.feature_importances_)

[0.09500419 0.88460211 0.0203937 ]


### XGBoost(Extreme Gradient Boosting)

- 히스토그램 기반 gbm을 분산환경에서도 실행할 수 있도록 구현한 알고리즘
- 기존 gbm에서 입력 특성을 256개의 구간으로 나눠 node를 분할할 떄 최적의 분할을 빠르게 찾을 수 있게 하는 알고리즘
- overfitting을 억제 하면서 기존 gbm보다 높은 성능을 제공함
- 기존 gbm의 경우 규제(regularazation) 기능이 없으나 XGBoost에는 자체적으로 규제 기능이 있다
- CART(Classification and regression tree) 앙상블 모델을 사용하기 때문에 분류와 회귀영역에서 높은 정확도를 보인다.
- hyperparameter tuning이 필수적이다

In [136]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [137]:
hgb = HistGradientBoostingClassifier(random_state=42)

In [138]:
hgb.fit(X_train,y_train)

HistGradientBoostingClassifier(random_state=42)

In [141]:
scores = cross_validate(rf,X_train,y_train,return_train_score=True,n_jobs=-1)

In [142]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [143]:
from sklearn.inspection import permutation_importance
hgb.fit(X_train,y_train)
result= permutation_importance(hgb,X_train,y_train,n_repeats=20,n_jobs=-1,random_state=42)

In [144]:
print(result.importances_mean)

[0.08831056 0.23558784 0.08093131]


In [145]:
from sklearn.inspection import permutation_importance
hgb.fit(X_train,y_train)
result= permutation_importance(hgb,X_test,y_test,n_repeats=20,n_jobs=-1,random_state=42)

In [146]:
print(result.importances_mean)

[0.05980769 0.20434615 0.04830769]


In [147]:
from xgboost import XGBClassifier

In [148]:
xgb= XGBClassifier(tree_method='hist',random_state=42)

In [149]:
scores = cross_validate(xgb,X_train,y_train,return_train_score=True)

[16:48:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:48:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:48:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:48:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [150]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


### LightGBM

![leaf](https://entheoscientist.files.wordpress.com/2020/04/e37e7-1azssoxb8lc5n6mnhqx5jcg.png)

![level](https://entheoscientist.files.wordpress.com/2020/04/b9f48-1whsa8ry4sgfqj1recwr8ag.png)

- tree가 수직적으로 확장되는 다른 tree 알고리즘들과 달리 tree가 수평적으로 확장되는 gbm 알고리즘

- 즉 level이 아닌 leaf를 확장하는 알고리즘(leaf-wise)이며 확장하기 위해 가장 큰 손실 함수를 가지는 leaf를 선택한다.

- 기존 level-wise 방식의 tree 알고리즘보다 더 많은 loss를 줄일 수 있는 장점이 있다.

- 속도가 빠르고 큰 사이즈의 데이터를 다루기 용이하기 때문에 가장 인기 있는 boosting 알고리즘 중 하나이다.

- 과적합에 민감하고 또한 데이터 셋이 작을수록 과적합하기 쉽기 때문에 작은 데이터 셋에서는 사용하기 어렵다

- parameter tuning이 필수적이다.

### LightGBM parameter

#### 파라미터 소개

- max_depth : Tree의 최대 깊이를 말합니다. 이 파라미터는 모델 과적합을 다룰 때 사용됩니다. 만약 여러분의 모델이 과적합된 것 같다고 느끼신다면 제 조언은 max_depth 값을 줄이라는 것입니다.

- min_data_in_leaf : Leaf가 가지고 있는 최소한의 레코드 수입니다. 디폴트값은 20으로 최적 값입니다. 과적합을 해결할 때 사용되는 파라미터입니다.

- feature_fraction : Boosting (나중에 다뤄질 것입니다) 이 랜덤 포레스트일 경우 사용합니다. 0.8 feature_fraction의 의미는 Light GBM이 Tree를 만들 때 매번 각각의 iteration 반복에서 파라미터 중에서 80%를 랜덤하게 선택하는 것을 의미합니다.

- bagging_fraction : 매번 iteration을 돌 때 사용되는 데이터의 일부를 선택하는데 트레이닝 속도를 높이고 과적합을 방지할 때 주로 사용됩니다.

- early_stopping_round : 이 파라미터는 분석 속도를 높이는데 도움이 됩니다. 모델은 만약 어떤 validation 데이터 중 하나의 지표가 지난 early_stopping_round 라운드에서 향상되지 않았다면 학습을 중단합니다. 이는 지나친 iteration을 줄이는데 도움이 됩니다.

- lambda : lambda 값은 regularization 정규화를 합니다. 일반적인 값의 범위는 0 에서 1 사이입니다.

- min_gain_to_split : 이 파라미터는 분기하기 위해 필요한 최소한의 gain을 의미합니다. Tree에서 유용한 분기의 수를 컨트롤하는데 사용됩니다.

- max_cat_group : 카테고리 수가 클 때, 과적합을 방지하는 분기 포인트를 찾습니다. 그래서 Light GBM 알고리즘이 카테고리 그룹을 max_cat_group 그룹으로 합치고 그룹 경계선에서 분기 포인트를 찾습니다. 디폴트 값은 64 입니다.

__Core Parameters
핵심 파라미터 소개__

- Task : 데이터에 대해서 수행하고자 하는 임무를 구체화합니다. train 트레이닝일 수도 있고 predict 예측일 수도 있습니다.

- application : 가장 중요한 파라미터로, 모델의 어플리케이션을 정하는데 이것이 regression 회귀분석 문제인지 또는 classification 분류 문제인지를 정합니다. Light GBM에서 디폴트는 regression 회귀분석 모델입니다.

- regression: 회귀분석
- binary: 이진 분류
- multiclass: 다중 분류
- boosting : 실행하고자 하는 알고리즘 타입을 정의합니다. 디폴트값은 gdbt 입니다.

- gdbt : Traditional Gradient Boosting Decision Tree
- rf : Random Forest
- dart : Dropouts meet Multiple Additive Regression Trees
- goss : Gradient-based One-Side Sampling
- num_boost_round : boosting iteration 수로 일반적으로 100 이상입니다.

- learning_rate : 최종 결과에 대한 각각의 Tree에 영향을 미치는 변수입니다. GBM은 초기의 추정값에서 시작하여 각각의Tree 결과를 사용하여 추정값을 업데이트 합니다. 학습 파라미터는 이러한 추정에서 발생하는 변화의 크기를 컨트롤합니다. 일반적인 값은 0.1, 0.001, 0.003 등등이 있습니다.

- num_leaves : 전체 Tree의 leave 수 이고, 디폴트값은 31입니다.

- device : 디폴트 값은 cpu 인데 gpu로 변경할 수도 있습니다.

__Metric parameter__
__지표 파라미터__

- metric : 모델을 구현할 때 손실을 정하기 때문에 중요한 변수 중에 하나입니다. regression과 classification 을 위한 일반적인 손실 값이 아래에 나와있습니다.

- mae : mean absolute error
- mse : mean squared error
- binary_logloss : loss for binary classification
- multi_logloss : loss for multi classification  

__IO parameter__

__IO 파라미터__

- max_bin : feature 값의 최대 bin 수를 의미합니다.

- categorical_feature : 범주형 feature의 인덱스를 의미합니다. 만약 categorical_features 가 0, 1, 2 이면 column 0, column 1, column 2 가 범주형 변수들입니다.

- ignore_column : categorical_features와 동일한 것인데 범주형 feature로써 특정 칼럼을 고려하지 않는 것입니다. 그 변수들을 무시하는 것입니다.

- save_binary : 데이터 파일의 메모리 사이즈를 처리해야 한다면 이 파라미터 값을 True로 설정하십시오. 이 값을 True로 세팅하면 데이터 세트를 바이너리 파일로 저장할 것이고, 이 바이너리 파일은 다음에 데이터를 읽어올 때 그 속도를 줄여줄 것입니다.

__튜닝 팁__

- num_leaves 는 개별 트리가 가질 수 있는 최대 리프의 개수를 지정합니다. (LightGBM 복잡도를 제어하는 주 파라미터) num_leaves가 클수록 모델의 깊이가 깊어지므로 모델 정확도가 높아질 수 있지만 과적합 문제가 발생할 확률 또한 높아집니다.
- min_data_in_leaf(min_child_in_leaf)는 과적합 개선을 위해 사용되는 주 파라미터로 큰 값을 설정하는 경우 트리가 깊어지는 것을 방지합니다.
- max_depth는 명시적으로 트리의 깊이를 제어하는 파라미터 입니다.


In [152]:
from lightgbm import LGBMClassifier

In [153]:
lg = LGBMClassifier(random_state=42)

In [154]:
scores = cross_validate(lg,X_train,y_train,return_train_score=True,n_jobs=-1)

In [155]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


### 서포트 벡터 머신(Suppot Vector Machine)

![svm](https://miro.medium.com/max/1162/1*-4FOSXGyV6CSUOrOPEFc9g.png)

- 서포트 벡터머신(SVM)이란 결정경계, 즉 분류를 위한 기준선을 정의하는 모델이다. 그래서 분류되지 않은 새로운 점이 나타나면 경계의 어느 쪽에 속하는지 확인해 분류를 수행할 수 있게 된다.

- 결정 경계란 위의 그림에서 서포트 벡터를 나누는 기준선이다.

- 위의 서포트 벡터의 속성(sample)이 2개면 2차원 평면 3개면 3차원 공간..으로 늘어 나는데 결정 경계가 고차원으로 생성되면 이를 __초평면(hyperplane)__ 이라고 부른다.

__마진(margin)__

- 마진(margin)은 결정경계와 서포트 벡터 사이의 거리를 의미한다.
- n개의 속성을 가진 데이터에는 최소 n+1개의 서포트 벡터가 존재한다.
- 최적의 결정 경계는 마진이 최대화일 때를 의미한다.
- 대부분의 머신러닝 학습 알고리즘과는 달리 svm에서는 데이터 포인트중 서포트 벡터만 잘 골라내면 나머지 쓸모없는 수많은 데이터 포인트들을 무시하고 학습할 수 있기 때문에 속도가 매우 빠르다

![마진](https://hleecaster.com/wp-content/uploads/2020/01/svm06-1024x768.png)

**하드 마진(hard margin)**

- 하드 마진이란 결정경계와 서포트 벡터 사이의 거리가 매우 좁은 마진(즉, 이상치를 허용하지 않음)을 뜻한다.
- 마진이 매우 작기 때문에 이상치(outlier)에 매우 민감하며 데이터가 선형적으로 구분될 수 있어야 제대로 동작하며 과적합이 발생할 수 있다.

**소프트 마진(soft margin)**

- 하드 마진과는 달리 이상치(outlier)들을 어느 정도 허용하는 마진
- 마진을 최대화 하여 일반화 성능을 높이고(Large Margin) 마진을 벗어날 수 있는 범위를 허용함으로써 모델의 유연성을 높인다(Flexible)
- 마진이 너무 크기 때문에 과소적합(undefitting)이 발생할 수도 있다.